This is a revised and re-organized edition of the core (non-trivial) work done in notebook, "Capstone Project 1 - Data Cleaning". See also, **"RunPreproc.py"** for the automated version of this notebook.

# setup

In [ ]:
# script for standard imports and reading gtd_df.csv
%run '/content/drive/MyDrive/Data_Science/scripts/setup_get_gtd_data.ipynb'

In [ ]:
%who DataFrame

df	 


In [ ]:
x = df.copy()
x.shape

(191464, 135)

# **Step 1: Initial Data Adjustments** 
(num/cat only)

In [ ]:
# get all DB admin encoded columns
admin_enc_df = \
x.loc[:, [c for c in 
        [c[:-4] for c in x if '_txt' in c]
        if c in x]]

#save for comparison or later usage
admin_enc_col_list = list(admin_enc_df.columns)
# build admin_enc_col_df
admin_enc_col_df = x.loc[:, admin_enc_col_list]

# do the same w/ lat and long 
# (add back to dataset later if wanted)
lat_long_df=x.loc[:, ['latitude','longitude']]

# combine into single column (for dt) create
'''to use: .astype('datetime') on "datetime_col",
need to fill '0' first at all locs'''

datetime_col = x['iday'].astype('string') + '-' \
+ x['imonth'].astype('string') + '-' \
+ x['iyear'].astype('string')

# remove original day,month,year cols
x = x.drop(['iday','imonth','iyear'], axis=1)

""" STEP #1 : Data-Type Adjustments """
# # Adjust to string Dtypes for text
txt_col_list = \
['motive', 'summary','location','propcomment']

x.loc[:, txt_col_list] = \
x.loc[:, txt_col_list].astype('string')

x['d_m_y'] = datetime_col

eventid_related_df = \
x.loc[:, ['eventid', 'related', 'dbsource']]

# """Get "cols_for_later" : """
cols_for_later = pd.concat(
    [x['d_m_y'], lat_long_df, eventid_related_df], axis=1)

1.0.1 : drop admin encoded cols and additional cols (for later analysis)

In [ ]:
# drop admin encoded cols
x = x.drop(list(admin_enc_col_list), axis=1)
# Drop additional cols (for later analysis)
x = x.drop(list(cols_for_later),axis=1)

In [ ]:
x.shape

(191464, 99)

1.0.2 : Get X train/test sets
- Because our target/response/dependent feature, `"success"`, is of imbalanced nature, we must to maintain the ratio among binary inputs ("0"/"1") while going about establishing our subsets of train and test sets with [`StratifiedShuffleSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html#sklearn-model-selection-stratifiedshufflesplit).



In [ ]:
data = x.copy()

SSS = StratifiedShuffleSplit(n_splits=1,
                              test_size=0.2,
                              random_state=3)
 
for train_idx, test_idx in SSS.split(data, 
                                     data['success']):
    # train and test sets
    train_x = data.iloc[train_idx, :]
    test_x = data.iloc[test_idx, :]

In [ ]:
train_x.shape, test_x.shape

((153171, 99), (38293, 99))

1.0.3 : Remove rows/columns with very high NaN-counts

In [ ]:
#remove rows/columns with very high NaN-counts
trx_nan_perc = train_x.isnull().sum() / len(train_x)

#condition
thresh=0.95
thresh_nans = trx_nan_perc>=thresh

#subset
print(f'Training set (trainx) contains \
{len(trx_nan_perc[thresh_nans])} variables with >= {thresh}% NaNs\n')

# view cols w/ large % Nans
high_nan_vars = trx_nan_perc[thresh_nans]
high_nan_vars.sort_values(ascending=False)

# remove cols w/ high NaN percentages from train set
# (save for later for other NaN-alternatives after initial run)
train_x.loc[:, list(high_nan_vars.index)]\
.to_csv('/content/drive/MyDrive/Data_Science/data/gtd/trainx_high_nans.csv')

# Remove Columns with large percentage of NaNs
train_x = train_x.drop(list(high_nan_vars.index),axis=1)

# trainx updated
print(f'"train_x" shape after removing columns w/ >= 95% NaNs: {train_x.shape}')

Training set (trainx) contains 33 variables with >= 0.95% NaNs

"train_x" shape after removing columns w/ >= 95% NaNs: (153171, 66)


1.0.4 : Establish X/y sets


In [ ]:
# establish "target"
trainy = train_x["success"]
testy = test_x['success']

# and drop from X sets
trainx = train_x.drop('success', axis=1)
testx = test_x.drop('success', axis=1)

In [ ]:
# view train/test sets for target
trainy.shape, testy.shape

((153171,), (38293,))

In [ ]:
#view X-sets after target removal
trainx.shape, trainy.shape

((153171, 65), (153171,))

In [ ]:
#before dtype adjustments
display(pd.Series(trainx.dtypes).value_counts())
trainx.info(verbose=False)

object     30
float64    17
int64      14
string      4
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153171 entries, 59924 to 95773
Columns: 65 entries, approxdate to INT_ANY
dtypes: float64(17), int64(14), object(30), string(4)
memory usage: 77.1+ MB


1.0.5 : optimize optimize memory/storage (training set)


In [ ]:
# Reminder to back astype("U") b4 tfidf
trx_txt = trainx.select_dtypes(include='string') 
trx_non_txt = trainx.select_dtypes(exclude='string')
# optimize object-types (categorical)
trx_cats = trx_non_txt.select_dtypes(object)
trx_cats_opt = optimize_objects(trx_cats)
# optimize int-types
trx_ints = trx_non_txt.select_dtypes(int)
trx_ints_opt = optimize_ints(trx_ints)
# optimize float-types
trx_floats = trx_non_txt.select_dtypes(float)
trx_floats_opt = optimize_floats(trx_floats)

trx_opt = pd.concat([trx_txt, trx_ints_opt,
                     trx_floats_opt, 
                     trx_cats_opt], axis=1)

In [ ]:
# trainx after dtype adjustments, "trx_opt"
display(pd.Series(trx_opt.dtypes).value_counts())
trx_opt.info(verbose=False)

object      29
float32     17
int8        14
string       4
category     1
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153171 entries, 59924 to 95773
Columns: 65 entries, location to scite3
dtypes: category(1), float32(17), int8(14), object(29), string(4)
memory usage: 53.3+ MB


> _We've decreased the memory usage by about 24MB after dtype-optimization._

In [ ]:
# without affecting the inherent training data:
set(trx_opt)==set(trainx)

True

> **NOTE: `trainx` == `trx_opt` ... where the latter is simply an optimized (memory/storage) version of the former**

In [ ]:
# write initial x/y splits, and optimized trainx, "trx_opt", to csv:
trainx.to_csv('/content/drive/MyDrive/Data_Science/data/gtd/trainx.csv',
            index=False)
trainy.to_csv('/content/drive/MyDrive/Data_Science/data/gtd/trainy.csv',
            index=False)
testx.to_csv('/content/drive/MyDrive/Data_Science/data/gtd/testx.csv',
            index=False)
testy.to_csv('/content/drive/MyDrive/Data_Science/data/gtd/testy.csv',
            index=False)
# "trx_opt", to csv:
trx_opt.to_csv('/content/drive/MyDrive/Data_Science/data/gtd/trx_opt.csv',
               index=False)

# **Step 2 : Preprocessing** - _Expanding on initial data adjustments_

2.0.1
- `replace_uppr_w_lower`

    - correcting database entry errors of missing data

2.0.2
- `merge_items`

    - correcting database entry errors extended: merge "equivilent" representation for "uncertain (i.e.; "-9" == "-99")

2.0.3
- `BuildBinaries`

    - missing data imputation with indicator or flag variable creation

2.0.4
- `select_cols_for_pipe`
    - creating explicit lists per dtype for sklearn pipeline (next notebook, "transformations")

In [ ]:
# move to current directory like magic 
%cd /content/drive/MyDrive/Data_Science/modules/gtd/

# load custom functions 
%load replace_uppr_w_lowr.ipynb
%load merge_items.ipynb
%load BuildBinaries.ipynb
%load select_cols_for_pipe.ipynb

# alternative; w/ full path name:
# >>> %run '/content/drive/MyDrive/Data_Science/modules/gtd/replace_uppr_w_lowr.ipynb'
# >>> %run '/content/drive/MyDrive/Data_Science/modules/gtd/merge_items.ipynb'
# >>> %run '/content/drive/MyDrive/Data_Science/modules/gtd/BuildBinaries.ipynb'
# >>> %run '/content/drive/MyDrive/Data_Science/modules/gtd/select_cols_for_pipe.ipynb'

In [ ]:
# determined during initial data exploration
missing_labels = ['Unknown',-9,-99]
make_lower = 'Unknown'
merge_labels = True
merge_list = [-9,-99]

2.0.1 : `replace_uppr_w_lowr`

In [ ]:
# pass trx_opt
trx_replaced, missing_labels_replaced = \
replace_uppr_w_lowr(x=trx_opt,
                    missing_labels=missing_labels,
                    make_lower=make_lower)

print(f'After "replace_ippr_w_lowr":\n\
x-train shape: {trx_replaced.shape}\nupdated list: {missing_labels_replaced}')

After "replace_ippr_w_lowr":
x-train shape: (153171, 65)
updated list: ['unknown', -9, -99]


2.0.2 : `merge_items`

In [ ]:
# pass trx_replaced, missing_labels_replaced
trx_replaced_merged, missing_labels_merged = \
merge_items(x=trx_replaced,
            missing_labels=missing_labels_replaced,
            merge_labels=merge_labels,
            merge_list=merge_list)

print(f'\n\nAfter "merge_items":\nx-train shape:\
{trx_replaced_merged.shape}\nmerged list: {missing_labels_merged}')

2.0.3 : `BuildBinaries`

In [ ]:
#pass trx_replaced_merged, missing_labels_merged
trx_preproc = \
BuildBinaries(x=trx_replaced_merged,
              missing_labels=missing_labels_merged)

print(f'\n\nAfter "BuildBinaries":\n\x-train shape:\n{trx_preproc.shape}')

In [ ]:
# write preprocessed working data-set (num/cat only), "trx_preproc", to csv
trx_preproc.to_csv('/content/drive/MyDrive/Data_Science/data/gtd/trx_preproc.csv',
                   index=False)

# Next Steps

Next, Establish baseline scores _(see "baseline scores after preproc.ipynb")._